In [90]:
import pandas as pd
import sqlite3
import numpy as np
import matrix_factorization_utilities

In [91]:
conn=sqlite3.connect("db.sqlite3")
#load user ratings
raw_dataset_df=pd.read_sql("select * from reviews;",conn)
#convert user ratings into a matrix
ratings_df=pd.pivot_table(raw_dataset_df,index='userid',columns='movieid',aggfunc=np.max)
#load movie titles
movies_df=pd.read_sql("select * from blog_movie;",conn,index_col="id")


In [92]:
raw_dataset_df

,userid,movieid,rating
0,1,3,3
1,1,2,4
2,1,4,5
3,1,5,1
4,7,2,2
5,7,4,5
6,8,3,2
7,8,4,2
8,8,5,4
9,9,5,1


In [93]:
ratings_df

rating               
movieid      2    3    4    5
userid                       
1          4.0  3.0  5.0  1.0
7          2.0  NaN  5.0  NaN
8          NaN  2.0  2.0  4.0
9          3.0  NaN  NaN  1.0
10         NaN  5.0  NaN  1.0

In [94]:
movies_df

,title,content,creator_id
id,,,
2,Padta PEC,https://www.youtube.com/embed/J-CSEWFDllM,1
3,L.S.D.,https://www.youtube.com/embed/kOGU1znMNrI,1
4,Aalsi Sharma,https://www.youtube.com/embed/5zQ7txPKmog,1
5,CIRCUIT,https://www.youtube.com/embed/Qk16dvmBx60,1


In [95]:
movies_df.columns

Index(['title', 'content', 'creator_id'], dtype='object')

In [96]:
U,M=matrix_factorization_utilities.low_rank_matrix_factorization(ratings_df.as_matrix(),num_features=15,regularization_amount=0.1)

/home/ryzbaka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Optimization terminated successfully.
         Current function value: 1.960480
         Iterations: 2104
         Function evaluations: 3301
         Gradient evaluations: 3301


In [97]:
predicted_ratings=np.matmul(U,M)

In [98]:
print("Enter a user id to get recommendations")
user_id_to_search=int(input())
print(f"Movies previously reviewed by user {user_id_to_search}")
reviewed_movies_df=raw_dataset_df[raw_dataset_df['userid']==user_id_to_search]
reviewed_movies_df=reviewed_movies_df.join(movies_df,on="movieid")

Enter a user id to get recommendations
10
Movies previously reviewed by user 10


In [99]:
print(reviewed_movies_df[['movieid','title','rating']])

    movieid    title  rating
11        3   L.S.D.       5
12        5  CIRCUIT       1


In [100]:
movies_df.index

Int64Index([2, 3, 4, 5], dtype='int64', name='id')

Okay so I should keep the movies_df just as it is and just movie_df.index instead of movies_df.id

In [101]:
already_reviewed=reviewed_movies_df['movieid']

In [102]:
already_reviewed

11    3
12    5
Name: movieid, dtype: int64

In [103]:
reviewed_movies_df

,userid,movieid,rating,title,content,creator_id
11,10,3,5,L.S.D.,https://www.youtube.com/embed/kOGU1znMNrI,1
12,10,5,1,CIRCUIT,https://www.youtube.com/embed/Qk16dvmBx60,1


In [104]:
#recommended_df=movies_df[movies_df.index not in list(reviewed_movies_df['movieid'])]
reviewed=list(reviewed_movies_df['movieid'])

In [105]:
reviewed

[3, 5]

In [106]:
movielist=list(movies_df.index)

In [107]:
movielist

[2, 3, 4, 5]

In [108]:
recommended_df=movies_df[movies_df.index.isin(reviewed) == False]

In [109]:
recommended_df

,title,content,creator_id
id,,,
2,Padta PEC,https://www.youtube.com/embed/J-CSEWFDllM,1
4,Aalsi Sharma,https://www.youtube.com/embed/5zQ7txPKmog,1


These aren't actual review based recommendations they are just movies that he hasn't watched so lets add the numerical part

In [110]:
predicted_ratings

array([[3.93740152, 3.00018085, 4.9571873 , 1.00641858],
       [2.02956097, 4.2467341 , 4.90978372, 0.88037057],
       [1.31059868, 1.99042447, 1.99725691, 3.90082317],
       [2.92788739, 1.09566491, 2.63632618, 0.99440028],
       [1.30145555, 4.90053909, 5.01522774, 1.00953213]])

In [111]:
ratings_df

rating               
movieid      2    3    4    5
userid                       
1          4.0  3.0  5.0  1.0
7          2.0  NaN  5.0  NaN
8          NaN  2.0  2.0  4.0
9          3.0  NaN  NaN  1.0
10         NaN  5.0  NaN  1.0

In [112]:
user_ratings=predicted_ratings[user_id_to_search-6]

In [113]:
user_ratings

array([1.30145555, 4.90053909, 5.01522774, 1.00953213])

In [114]:
movies_df['ratings']=user_ratings

In [115]:
movies_df

,title,content,creator_id,ratings
id,,,,
2,Padta PEC,https://www.youtube.com/embed/J-CSEWFDllM,1,1.301456
3,L.S.D.,https://www.youtube.com/embed/kOGU1znMNrI,1,4.900539
4,Aalsi Sharma,https://www.youtube.com/embed/5zQ7txPKmog,1,5.015228
5,CIRCUIT,https://www.youtube.com/embed/Qk16dvmBx60,1,1.009532


In [116]:
reviewed=list(reviewed_movies_df['movieid'])

In [117]:
recommended_df=movies_df[movies_df.index.isin(reviewed) == False]

In [118]:
recommended_df

,title,content,creator_id,ratings
id,,,,
2,Padta PEC,https://www.youtube.com/embed/J-CSEWFDllM,1,1.301456
4,Aalsi Sharma,https://www.youtube.com/embed/5zQ7txPKmog,1,5.015228


In [119]:
recommended_df=recommended_df.sort_values(by=['ratings'],ascending=False)

In [120]:
recommended_df.head()

,title,content,creator_id,ratings
id,,,,
4,Aalsi Sharma,https://www.youtube.com/embed/5zQ7txPKmog,1,5.015228
2,Padta PEC,https://www.youtube.com/embed/J-CSEWFDllM,1,1.301456


So the problem is that the recommendation system is suggesting all these horrible movies because there simply aren't that many of them and the recommendation system is supposed to return the top 5 most likely to get high rating movies when i only have 5 movies out of which the user has already watched a few lol

Update the main make_recommendations.py program with the stuff that you solved here alright? 

and drink some water goddamit.